# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [2]:
from ib_async import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 4002, clientId=15)

<IB connected to 127.0.0.1:4002 clientId=15>

### Streaming tick data

Create some Forex contracts:

In [3]:
contracts = [Forex(pair) for pair in ('EURUSD', 'USDJPY', 'GBPUSD', 'USDCHF', 'USDCAD', 'AUDUSD')]
ib.qualifyContracts(*contracts)

eurusd = contracts[0]

Request streaming ticks for them:

In [4]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [5]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2024, 7, 10, 19, 17, 23, 310626, tzinfo=datetime.timezone.utc), minTick=1e-05, bid=1.08249, bidSize=1000000.0, ask=1.0825, askSize=3500000.0, prevAskSize=2500000.0, high=1.0829, low=1.08105, close=1.0814)

The price of Forex ticks is always nan. To get a midpoint price use ``midpoint()`` or ``marketPrice()``.

The tickers are kept live updated, try this a few times to see if the price changes:

In [9]:
ticker.marketPrice()

1.0826

The following cell will start a 30 second loop that prints a live updated ticker table.
It is updated on every ticker change.

In [11]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[c.pair() for c in contracts],
    columns=['bidSize', 'bid', 'ask', 'askSize', 'high', 'low', 'close'])

def onPendingTickers(tickers):
    for t in tickers:
        df.loc[t.contract.pair()] = (
            t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers

,bidSize,bid,ask,askSize,high,low,close
EURUSD,2000000.0,1.08254,1.08255,1500000.0,1.0829,1.08105,1.0814
USDJPY,5000000.0,161.763,161.769,6000000.0,161.805,161.26,161.32
GBPUSD,1000000.0,1.28409,1.28411,2000000.0,1.2847,1.2784,1.2786
USDCHF,29000.0,0.8997,0.89972,2000000.0,0.9001,0.89645,0.89785
USDCAD,2000000.0,1.36235,1.36238,3000000.0,1.3643,1.3608,1.3635
AUDUSD,2000000.0,0.67426,0.67428,1000000.0,0.67525,0.6732,0.6741


New tick data is available in the 'ticks' attribute of the pending tickers.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [12]:
for contract in contracts:
    ib.cancelMktData(contract)

### Tick by Tick data ###

The ticks in the previous section are time-sampled by IB in order to cut on bandwidth. So with ``reqMktdData`` not every tick from the exchanges is sent. The promise of ``reqTickByTickData`` is to send every tick, just how it appears in the TWS Time & Sales window. This functionality is severly nerfed by a total of just three simultaneous subscriptions, where bid-ask ticks and sale ticks also use up a subscription each.

The tick-by-tick updates are available from ``ticker.tickByTicks`` and are signalled by ``ib.pendingTickersEvent`` or ``ticker.updateEvent``.

In [13]:
ticker = ib.reqTickByTickData(eurusd, 'BidAsk')
ib.sleep(2)
print(ticker)

ib.cancelTickByTickData(ticker.contract, 'BidAsk')

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2024, 7, 10, 19, 34, 0, 281589, tzinfo=datetime.timezone.utc), minTick=1e-05, bid=1.08259, bidSize=2500000.0, ask=1.08261, askSize=4000000.0, prevBid=1.08254, prevBidSize=3000000.0, prevAsk=1.08256, prevAskSize=4500000.0, high=1.0829, low=1.08105, close=1.0814, tickByTicks=[TickByTickBidAsk(time=datetime.datetime(2024, 7, 10, 19, 34, 0, 281589, tzinfo=datetime.timezone.utc), bidPrice=1.08259, askPrice=1.08261, bidSize=2500000.0, askSize=4000000.0, tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False))])


### Historical tick data

Historical tick data can be fetched with a maximum of 1000 ticks at a time. Either the start time or the end time must be given, and one of them must remain empty:

In [15]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks[-1]

HistoricalTickBidAsk(time=datetime.datetime(2024, 7, 10, 19, 35, 15, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False), priceBid=1.08259, priceAsk=1.0826, sizeBid=2000000.0, sizeAsk=2500000.0)

In [16]:
ib.disconnect()